In [ ]:
import pandas as pd

# 1. 設定資料集路
github_raw_url = 'https://raw.githubusercontent.com/shu0518/hw4_m11423036/refs/heads/main/%E4%BA%A4%E6%98%93%E8%B3%87%E6%96%99%E9%9B%86.csv'
file_name = '交易資料集.csv'
quantity_col = 'QUANTITY'  # <---【重要】請改為您資料集中代表「數量」的欄位名稱 (例如 '數量' 或 'QTY')

# 2. 處理函式
def load_and_process(url, qty_col_name):
    # --- [讀取] ---
    df = None
    encodings = ['utf-8', 'big5', 'cp950', 'gbk']
    for fmt in encodings:
        try:
            df = pd.read_csv(url, encoding=fmt)
            break
        except UnicodeDecodeError:
            continue

    if df is None:
        print("讀取失敗，請檢查網址或編碼。")
        return

    # 紀錄原始筆數
    raw_count = len(df)
    print(f"正在處理資料集：{file_name} ...\n")

    # --- [前處理] ---

    # 1. 欄位名稱去除空白
    df.columns = [col.strip() for col in df.columns]

    # 2. 剔除退貨與註銷 (數量 <= 0)
    removed_returns_count = 0
    if qty_col_name in df.columns:
        # 確保該欄位是數字型態
        df[qty_col_name] = pd.to_numeric(df[qty_col_name], errors='coerce')

        # 執行篩選 (保留 > 0)
        initial_rows = len(df)
        df = df[df[qty_col_name] > 0].copy()
        removed_returns_count = initial_rows - len(df)
    else:
        print(f"⚠️ 警告：找不到欄位 '{qty_col_name}'，無法執行退貨剔除。\n")

    # 3. 日期欄位轉換格式 (TRX_DATE)
    if 'TRX_DATE' in df.columns:
        df['TRX_DATE'] = pd.to_datetime(df['TRX_DATE'], errors='coerce')

    # 4. 移除重複樣本
    before_dedup = len(df)
    df.drop_duplicates(inplace=True)
    removed_duplicates_count = before_dedup - len(df)

    # --- [輸出資料集資訊] ---
    final_count = len(df)

    print("="*50)
    print(f"【資料集資訊報告】")
    print(f"資料集名稱　　　：{file_name}")
    print("-" * 50)
    print(f"原始資料筆數　　：{raw_count}")
    print(f"剔除退貨/註銷 　：{removed_returns_count} (數量 <= 0)")
    print(f"移除重複筆數　　：{removed_duplicates_count}")
    print(f"最終有效筆數　　：{final_count}")
    print("-" * 50)

    print(f"【欄位與型態清單】(共 {len(df.columns)} 個欄位)")
    print(f"{'欄位名稱':<20} | {'資料型態'}")
    print("-" * 50)

    def get_zh_type(dtype):
        s = str(dtype)
        if 'int' in s: return '整數 (int)'
        if 'float' in s: return '浮點數 (float)'
        if 'datetime' in s: return '日期時間 (datetime)'
        return '文字/物件 (object)'

    for col in df.columns:
        print(f"{col:<20} | {get_zh_type(df[col].dtype)}")

    print("="*50)

    # --- [新增：顯示前 5 筆有效交易] ---
    print("\n【處理後資料前 5 筆預覽】")
    # 設定顯示參數，讓中文對齊更整齊
    pd.set_option('display.unicode.east_asian_width', True)
    pd.set_option('display.max_columns', None) # 顯示所有欄位
    pd.set_option('display.width', 1000)
    print(df.head(5))
    print("-" * 50)

    # 輸出檔案
    df.to_csv('交易資料集_processed.csv', index=False, encoding='utf-8-sig')
    print("\n已輸出檔案：交易資料集_processed.csv")

    return df

# 執行
df_clean = load_and_process(github_raw_url, quantity_col)

正在處理資料集：交易資料集.csv ...

【資料集資訊報告】
資料集名稱　　　：交易資料集.csv
--------------------------------------------------
原始資料筆數　　：157396
剔除退貨/註銷 　：28559 (數量 <= 0)
移除重複筆數　　：16876
最終有效筆數　　：111961
--------------------------------------------------
【欄位與型態清單】(共 7 個欄位)
欄位名稱                 | 資料型態
--------------------------------------------------
INVOICE_NO           | 文字/物件 (object)
CUST_ID              | 整數 (int)
ITEM_ID              | 整數 (int)
ITEM_NO              | 文字/物件 (object)
PRODUCT_TYPE         | 文字/物件 (object)
TRX_DATE             | 日期時間 (datetime)
QUANTITY             | 整數 (int)

【處理後資料前 5 筆預覽】
   INVOICE_NO  CUST_ID  ITEM_ID               ITEM_NO    PRODUCT_TYPE   TRX_DATE  QUANTITY
0  CX47348203     3218  3217532        M25P40-VMN6TPB  MEMORY_EMBEDED 2016-07-26      2500
1  CX47346522     2470  3326781  AU80610006237AASLBX9       CPU / MPU 2016-07-11        50
2  CX47348534    16135   740487          MMBD2837LT1G        DISCRETE 2016-07-27      3000
4  CX47346184     2356    70072          MMBT3